# BANK MARKETING

<br><br>
Membros:
- Anderson Jesus
- Caio Viera
- Pedro Correia



> CRIAÇÃO DE MODELO PREDITIVO

#### Inicializando sessão do Spark

In [1]:
import findspark
findspark.init('/home/labdata/spark-2.2.1-bin-hadoop2.6')

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName('bank').getOrCreate()

#### Carregando os Dados

In [3]:
data = spark.read.csv(
    "hdfs://elephant:8020/user/labdata/bank-historical-data.csv",
    header=True,
    sep=",",
    inferSchema=True
)

In [4]:
data = data.selectExpr(*["`{}` as {}".format(col, col.replace('.', '_')) for col in data.columns])

In [5]:
data.head()

Row(age=35, job='admin.', marital='single', education='university.degree', default='no', housing='yes', loan='no', contact='cellular', month='may', day_of_week='tue', duration=255, campaign=2, pdays=999, previous=1, poutcome='failure', emp_var_rate=-1.8, cons_price_idx=92.89299999999999, cons_conf_idx=-46.2, euribor3m=1.291, nr_employed=5099.1, y='no', client_id='TRZK1')

#### Preparação dos Dados

Definindo variáveis utilizadas pelo tipo a fim de realizar o encoding necessário

In [6]:
categoricalColumns = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'month',
    'day_of_week',
    'poutcome'
]

# não utilizaremos a variável `duration`, que algo não sabido antes da ligação ocorrer
# e portanto, não deve ser válida para fins preditivos
numericColumns = [
    'pdays',
    'previous',
    'emp_var_rate',
    'cons_price_idx',
    'cons_conf_idx',
    'euribor3m',
    'nr_employed'
]

Iniciando a construção do Pipeline

In [7]:
from pyspark.ml import Pipeline

In [8]:
# encoders e indexadores necessários ao tratamento dos dados
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

# instanciando nossa lista de passos a serem fornecidos ao pipeline
stages = []

Target

In [9]:
# indexação da variável resposta
indexer = StringIndexer(
    inputCol='y', 
    outputCol='label'
).setHandleInvalid("skip")

stages += [indexer]

Dados Categóricos

In [10]:
# transformações dados categóricos (paralelo pandas: get_dummies)
for categoricalCol in categoricalColumns:
    # nomes para valores [0:n_cats-1]
    indexer = StringIndexer(
        inputCol=categoricalCol, 
        outputCol=categoricalCol+'_index'
    ).setHandleInvalid("skip")
    # criando dummies
    encoder = OneHotEncoder(
        inputCol=categoricalCol+'_index',
        outputCol=categoricalCol+'_class_vec'
    )
    # inserindo estágios de transformação
    stages += [indexer, encoder]

Dados Numéricos

In [11]:
# transformando variáveis numéricas para o tipo double
for numericCol in numericColumns:
    data = data.withColumn(numericCol, data[numericCol].cast('double'))

Assembler

In [12]:
# criando assembler, que deixa os dados no formato vetorial 
# demandado pela biblioteca ML do Spark

assembler_inputs = [categoricalCol+'_class_vec' for categoricalCol in categoricalColumns]
assembler_inputs += numericColumns
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

stages += [assembler]

#### Modelagem

*Gradient Boosting Machine*

In [13]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(
    labelCol="label",
    featuresCol="features",
    predictionCol='prediction',
    maxIter=60,
    stepSize=0.01,
    seed=420
)

stages += [gbt]

In [14]:
pipeline = Pipeline(stages=stages)
pipeline.write().overwrite().save('hdfs://elephant:8020/user/labdata/model2/bank-pipeline-model-unfit')

#### Treinando e Validando Modelo

In [15]:
# separação de dados em treino e teste
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed=420)

print('Observações para treino: {:>7}'.format(trainingData.count()))
print('Observações para teste:  {:>7}'.format(testData.count()))

Observações para treino:   26389
Observações para teste:     6561


In [16]:
%%time
pipelineModel = pipeline.fit(trainingData)
pipelineModel.write().overwrite().save('hdfs://elephant:8020/user/labdata/model2/bank-pipeline-model')

CPU times: user 76.2 ms, sys: 0 ns, total: 76.2 ms
Wall time: 57.2 s


In [17]:
predictions_gbt = pipelineModel.transform(testData)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction",
    metricName="accuracy"
)

evaluator_auc = BinaryClassificationEvaluator(
    labelCol="label", 
    rawPredictionCol="rawPrediction"
)

accuracy_gbt = evaluator_accuracy.evaluate(predictions_gbt)
print('accuracy:         {:.4f}'.format(accuracy_gbt))
auc_gbt = evaluator_auc.evaluate(predictions_gbt)
print(f'areaUnderROC:     {auc_gbt:.4f}')

accuracy:         0.9032
areaUnderROC:     0.7881


In [19]:
predictions_gbt.select('label', 'prediction').createOrReplaceTempView('predictions')

spark.sql("""
SELECT
    round((tp+tn)/(tp+tn+fp+fn), 4) as accuracy,
    round(tp/(tp+fp), 4) as precision,
    round(tp/(tp+fn), 4) as recall
FROM (
    SELECT
        sum(tn) as tn,
        sum(tp) as tp,
        sum(fn) as fn,
        sum(fp) as fp
    FROM (
        SELECT
            case when label = 0 and prediction = 0 then 1 else 0 end as tn,
            case when label = 1 and prediction = 1 then 1 else 0 end as tp,
            case when label = 1 and prediction = 0 then 1 else 0 end as fn,
            case when label = 0 and prediction = 1 then 1 else 0 end as fp
        FROM
            predictions
    )
)
""").show()

+--------+---------+------+
|accuracy|precision|recall|
+--------+---------+------+
|  0.9032|   0.6656|0.2805|
+--------+---------+------+



`GBTClassifier` apresentou maior potencial, apesar de ter um recall ainda muito baixo. 

Na sequência, vamos buscar realizar um resampling da base, buscando deixar a classe positiva mais prevalente.

In [32]:
data_res = data.sampleBy('y', fractions={'yes': 1, 'no': 0.2}, seed=420)
data_res.groupBy('y').count().show()

+---+-----+
|  y|count|
+---+-----+
| no| 5886|
|yes| 3713|
+---+-----+



In [33]:
(trainingData, testData) = data_res.randomSplit([0.8, 0.2], seed=420)

print('Observações para treino: {:>7}'.format(trainingData.count()))
print('Observações para teste:  {:>7}'.format(testData.count()))

Observações para treino:    7686
Observações para teste:     1913


In [34]:
%%time
# criando novo modelo com resampling
pipelineModel_res = pipeline.fit(trainingData)

# salvando o modelo
pipelineModel_res.write().overwrite().save('hdfs://elephant:8020/user/labdata/model2/bank-pipeline-model-res')

CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 26.3 s


In [35]:
predictions_res = pipelineModel_res.transform(testData)

In [36]:
predictions_res.select('client_id', 'label', 'probability', 'prediction').show(10)

+---------+-----+--------------------+----------+
|client_id|label|         probability|prediction|
+---------+-----+--------------------+----------+
|    99E3E|  1.0|[0.15967127685075...|       1.0|
|    55UK3|  1.0|[0.11209598280032...|       1.0|
|    M7AJH|  1.0|[0.10728508532343...|       1.0|
|    3M8VA|  1.0|[0.63161893547033...|       0.0|
|    4621G|  0.0|[0.75710971442414...|       0.0|
|    QVAU3|  1.0|[0.90527359235893...|       0.0|
|    350UZ|  0.0|[0.71177995924388...|       0.0|
|    3SIFF|  1.0|[0.26985526483253...|       1.0|
|    28JND|  1.0|[0.41451071872129...|       1.0|
|    MQL5O|  0.0|[0.63226260473842...|       0.0|
+---------+-----+--------------------+----------+
only showing top 10 rows



In [37]:
accuracy_res = evaluator_accuracy.evaluate(predictions_res)
print('Accuracy:         {:.4f}'.format(accuracy_res))
auc_gbt = evaluator_auc.evaluate(predictions_gbt)
print(f'areaUnderROC:     {auc_gbt:.4f}')

Accuracy:         0.7669
areaUnderROC:     0.7881


In [38]:
predictions_res.select('label', 'prediction').createOrReplaceTempView('predictions_res')

spark.sql("""
SELECT
    round((tp+tn)/(tp+tn+fp+fn), 4) as accuracy,
    round(tp/(tp+fp), 4) as precision,
    round(tp/(tp+fn), 4) as recall
FROM (
    SELECT
        sum(tn) as tn,
        sum(tp) as tp,
        sum(fn) as fn,
        sum(fp) as fp
    FROM (
        SELECT
            case when label = 0 and prediction = 0 then 1 else 0 end as tn,
            case when label = 1 and prediction = 1 then 1 else 0 end as tp,
            case when label = 1 and prediction = 0 then 1 else 0 end as fn,
            case when label = 0 and prediction = 1 then 1 else 0 end as fp
        FROM
            predictions_res
    )
)
""").show()

+--------+---------+------+
|accuracy|precision|recall|
+--------+---------+------+
|  0.7669|   0.7786|0.5806|
+--------+---------+------+



Como esperávamos, o recall apresentou um valor bem mais interessante, demonstrando o potencial da solução, que certamente poderia ser refinada ainda mais a fim de atingir resultados mais expressivos.

<hr>